### Installing dependencies

Next, we install the necessary packages for PyTorch (`torch` and `torchvision`) and Flower (`flwr`):

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
!pip install beautifulsoup4
from bs4 import BeautifulSoup
from IPython.display import display, HTML, clear_output
from ipywidgets import Button, Layout
import threading
import ipywidgets as widgets
import IPython.display as display_output
from google.colab import drive 
drive.mount('/content/drive', force_remount=True) 
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt
!pip install -q flwr[simulation] torch torchvision matplotlib

from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
 
import flwr as fl
from flwr.common import Metrics
DEVICE = torch.device("cpu")
    # Mount Google Drive

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [ ]:
# Set the layout for the button
button_layout = widgets.Layout(width='auto')

# Create a label for printing the size of the data
size_label = widgets.Label()

mess='<h1 style="color:red; width:100px;" >____________WELLCOME____________</h1>'
display(HTML(mess))
cen='<h1 style="color:blue; width:100px;" >THE_FEDERATED_LEARNING_IN_SMART</h1>'
cenn='<h1 style="color:blue; width:100px;">____________HEALTHCARE___________</h1>'
display(HTML(cen))  
display(HTML(cenn)) 
# Create a label for the description of the dataset choice
description = widgets.Label(value='NUMBER_of_hospital:')
description_label = widgets.Label(value='Dataset Choice:')

# Create the widgets
num_clients_input = widgets.Dropdown(description='', options=[1, 5, 10, 100])
dataset_choice_dropdown = widgets.Dropdown(description='', options=['COVID_19', 'PNEUMONIA', 'Alzheimer_s', 'brian_tumor'])
button = widgets.Button(description='Load Datasets')
button = widgets.Button(description='Load Datasets')
# Function to load datasets and assign values to the global variables
def load_datasets(num_clients, dataset_choice):
    global num_c, trainloaders, valloaders, testloader, model, classes

    # Dataset paths
    dataset_paths = {
        'COVID_19': '/content/drive/MyDrive/data/dataset/COVID_19',
        'PNEUMONIA': '/content/drive/MyDrive/data/dataset/PNEUMONIA',
        'Alzheimer_s': '/content/drive/MyDrive/data/dataset/Alzheimer_s Dataset',
        'brian_tumor': '/content/drive/MyDrive/data/dataset/brian_tumor_dataset'
    }

    datatrain = f'{dataset_paths[dataset_choice]}/train'
    datatset = f'{dataset_paths[dataset_choice]}/test'
    BATCH_SIZE = 32
    img_size = (224, 224)

    # Download and transform MRI Brain Tumor Detector dataset
    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    trainset = ImageFolder(datatrain, transform=transform)
    testset = ImageFolder(datatset, transform=transform)
    size_label.value = f'The size of data: {len(trainset) + len(testset)}'

    # Get the classes from the trainset
    classes = trainset.classes
    num_c = len(classes)

    # Split training set into num_clients partitions to simulate the individual dataset
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader, classes,num_c

# Create the neural network model
class Net(nn.Module):
    def __init__(self, num_c):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 28 * 28, 264)
        self.fc2 = nn.Linear(264,num_c)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.pool3(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x


# Function to train the network
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")

# Function to test the network
def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

# Register the button callback
def button_clicked(b):
    global classes, trainloaders, valloaders, testloader, model, num_clients, num_c  # Use the global variables

    num_clients = num_clients_input.value
    dataset_choice = dataset_choice_dropdown.value
    load_datasets(num_clients, dataset_choice)

    # Create the neural network model
    model = Net(num_c)
    print("Number of classes:", num_c)
    print('num_clients',num_clients)
    print('Waiting for the plot to display...')

    # Create a dictionary to store one image from each class
    class_images = {}

    # Iterate over the trainloaders to get one image from each class
    for trainloader in trainloaders:
        for images, labels in trainloader:
            for image, label in zip(images, labels):
                class_name = classes[label]
                if class_name not in class_images:
                    # Reshape and convert the image to a NumPy array
                    image = image.permute(1, 2, 0).numpy()
                    # Denormalize
                    image = image / 2 + 0.5
                    class_images[class_name] = image
                    break

    # Create a figure and a grid of subplots
    fig, axs = plt.subplots(1, len(class_images), figsize=(5, 3))

    # Loop over the classes and plot the corresponding image
    for i, (class_name, image) in enumerate(class_images.items()):
        ax = axs[i]
        ax.imshow(image)
        ax.set_title(class_name)
        ax.axis("off")

    # Set the title of the plot
    fig.suptitle('Sample of Selected MRI Datasets', fontsize=14)

    # Show the plot
    plt.tight_layout()
    plt.rcParams['figure.dpi'] = 100

    # Create an output widget
    output_widget = widgets.Output()

    # Display the plot inside the output widget
    with output_widget:
        plt.show()

    # Clear the previous output, if any
    #display_output.clear_output(wait=True)

    # Display the output widget
    display_output.display(output_widget)
    
    # Create a label widget for the output
    
#///////////////////////
    

# Register the button callback
button.on_click(button_clicked)

# Display the widgets

display(description)
display(num_clients_input)
display(description_label)
display(dataset_choice_dropdown)
display(size_label)
display(button, layout=button_layout)




message = '<h3 style="color:red; width:100px;">NEXT STEP</h3>'
display(HTML(message))
cen = '<h1 style="color:blue; width:100px;">Centralized_Training</h1>'
display(HTML(cen))  
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create widgets
label_train_central = widgets.Label(value='Train Central')
button_yes = widgets.Button(description='Yes')
button_no = widgets.Button(description='No')
input_num_epochs = widgets.IntText(description='NUM_EPOCHS')
button_train = widgets.Button(description='Train', button_style='success')
output_train_epoch = widgets.Output()

# Define button click handlers
def on_button_yes_clicked(button):
    button_yes.layout.display = 'none'
    button_no.layout.display = 'none'
    input_num_epochs.layout.display = ''
    button_train.layout.display = 'flex'

def on_button_no_clicked(button):
    button_yes.layout.display = 'none'
    button_no.layout.display = 'none'
    button_train.layout.display = 'none'
    print('OK')

def on_button_train_clicked(button):
        trainloader = trainloaders[input_num_epochs.value]
        input_num_epochs.layout.display = 'none'
        button_train.layout.display = 'none'
        with output_train_epoch:
            clear_output()  # Clear previous output
        accuracy_history = []   
        for epoch in range(input_num_epochs.value):
            train(model, trainloader, 1)
            loss, accuracy = test(model, testloader)
            accuracy_history.append(accuracy)  # Append accuracy to history
            print(f"Epoch {epoch+1}: test loss {loss}, accuracy {accuracy}")
        message = '<h4 style="color:red; width:100px;">training is over</h4>'
        display(HTML(message))

# Register button click event handlers
button_yes.on_click(on_button_yes_clicked)
button_no.on_click(on_button_no_clicked)
button_train.on_click(on_button_train_clicked)

# Set initial widget visibility
input_num_epochs.layout.display = 'none'
button_train.layout.display = 'none'

# Display the widgets
display(label_train_central)
display(button_yes)
display(button_no)
display(input_num_epochs)
display(button_train)
display(output_train_epoch)











message = '<h3 style="color:red; width:100px;">NEXT STEP</h3>'
display(HTML(message))
cen = '<h1 style="color:blue; width:100px;">FEDRATED_LEARNIN_WITH_FLOWER</h1>'
display(HTML(cen))  



def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

def aggregate_FedAdagrad(metrics: List[Tuple[int, dict]]) -> dict:
    total_examples = sum([num_examples for num_examples, _ in metrics])
    weighted_sum = {key: np.zeros_like(metrics[0][1][key]) for key in metrics[0][1]}  # Initialize weighted sum with zeros

    for num_examples, metric in metrics:
        for key in metric:
            # Multiply each metric value by the number of examples and add to the weighted sum
            weighted_sum[key] += num_examples * metric[key]

    # Divide the weighted sum by the total number of examples to compute the weighted average
    weighted_avg = {key: weighted_sum[key] / total_examples for key in weighted_sum}

    return weighted_avg

penalty_weight = 0.1 
def aggregate_FedProx(metrics: List[Tuple[int, dict]]) -> dict:
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate accuracy and examples
    aggregated_metrics = {
        "accuracy": sum(accuracies) / sum(examples),
        "examples": sum(examples)
    }

    # Apply penalty term to the aggregated accuracy
    aggregated_metrics["accuracy_with_penalty"] = aggregated_metrics["accuracy"] + penalty_weight * aggregated_metrics["examples"]

    return aggregated_metrics

def average_FedAvg(metrics: List[Tuple[int, dict]]) -> dict:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

def aggregation_FedAdam(metrics: List[Tuple[int, dict]]) -> dict:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    total_examples = sum(examples)
    weighted_acc_sum = sum(accuracies)

    # Apply FedAdam strategy (e.g., update the weighted average with global update)
    global_update = 0.1  # Replace with the appropriate global update value

    weighted_acc_sum += global_update * total_examples
    total_examples += global_update

    return {"accuracy": weighted_acc_sum / total_examples}

#Using the Virtual Client Engine
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net(num_c).to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader)





# Define num_round_input dropdown
num_round_input = widgets.Dropdown(
    options=[1, 2, 5, 10, 100],
   description="N_Rounds:"


)

# Define strategy dropdown
strategy_dropdown = widgets.Dropdown(
    options=["FedAvg", "FedAdam", "FedProx", "FedAdagrad"],
    description="Strategy:"
 
)

# Define button
run_button = widgets.Button(description="Run Simulation")

# Define output text area
output_text = widgets.Output()

# Define function to run simulation on button click
def run_simulation(button):
    num_round = num_round_input.value
    
    if strategy_dropdown.value == "FedAvg":
        strategy = fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            fraction_evaluate=0.5,
            min_fit_clients=num_clients - 1,
            min_evaluate_clients=num_clients - 2,
            min_available_clients=num_clients,
            initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net(num_c))),
            evaluate_metrics_aggregation_fn=average_FedAvg,  # <-- pass the metric aggregation function
        )
        # Start simulation with FedAvg strategy
    elif strategy_dropdown.value == "FedAdagrad":
        # Start simulation with FedAdagrad strategy
        from flwr.server.strategy import FedAdagrad
        # Create FedAdagrad strategy
        strategy = FedAdagrad(
            fraction_fit=1.0,
            fraction_evaluate=0.5,
            min_fit_clients=num_clients - 1,
            min_evaluate_clients=num_clients - 2,
            min_available_clients=num_clients,
            initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net(num_c))),
            evaluate_metrics_aggregation_fn=aggregate_FedAdagrad,  
        )
    elif strategy_dropdown.value == "FedProx":
        # Start simulation with FedProx strategy
        from flwr.server.strategy.fedprox import FedProx
        strategy = FedProx(
            fraction_fit=1.0,
            fraction_evaluate=0.5,
            min_fit_clients=num_clients - 1,
            min_evaluate_clients=num_clients - 2,
            min_available_clients=num_clients,
            evaluate_metrics_aggregation_fn=aggregate_FedProx,
            initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net(num_c))),
            proximal_mu=0.1,
        )
    elif strategy_dropdown.value == "FedAdam":
        # Start simulation with FedAdam strategy
        from flwr.server.strategy import FedAdam
        strategy = FedAdam(
            fraction_fit=1.0,
            fraction_evaluate=0.5,
            min_fit_clients=num_clients - 1,
            min_evaluate_clients=num_clients - 2,
            min_available_clients=num_clients,
            evaluate_metrics_aggregation_fn=aggregation_FedAdam,
            initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net(num_c))),
            eta=1e-1,  # Adjust the learning rate if needed
            eta_l=1e-1,  # Adjust the client-side learning rate if needed
            beta_1=0.9,  # Adjust the momentum parameter if needed
            beta_2=0.99,  # Adjust the second moment parameter if needed
            tau=1e-9,
        )
    else:
        # Run simulation using other strategy
        # Add code for the other strategy here
        pass

    # Update with your specific simulation code
    
    output = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=num_round),
        strategy=strategy,
        client_resources=client_resources,
    )

    # Print output
    with output_text:
        print(output)

# Add button click event handler
run_button.on_click(run_simulation)

# Display widgets
display(widgets.VBox([num_round_input, strategy_dropdown, run_button, output_text]))

message = '<h3 style="color:red; width:100px;">THE_END</h3>'
display(HTML(message))
cen = '<h1 style="color:blue; width:100px;">THE_OUTPUT_CLASSES_OF_DATASET</h1>'
display(HTML(cen))  

Label(value='NUMBER_of_hospital:')

Dropdown(options=(1, 5, 10, 100), value=1)

Label(value='Dataset Choice:')

Dropdown(options=('COVID_19', 'PNEUMONIA', 'Alzheimer_s', 'brian_tumor'), value='COVID_19')

Label(value='')

...

Button(description='Yes', style=ButtonStyle())

Button(description='No', style=ButtonStyle())

IntText(value=0, description='NUM_EPOCHS', layout=Layout(display='none'))

Button(button_style='success', description='Train', layout=Layout(display='none'), style=ButtonStyle())

Output()